In [ ]:
# Function to identify participants with inconsistent responses
def identify_inconsistent_participants(df):
    """
    Identify participants with inconsistent responses:
    - Choose City B but estimate P(City A > City B) < 50%
    - Choose City A but estimate P(City A > City B) > 50%
    
    Returns: DataFrame with inconsistent participants and their details
    """
    inconsistent_participants = []
    
    # Get prediction task data only
    pred_data = df[df['trial_type'] == 'prediction-task'].copy()
    
    for participant_id in pred_data['participant_id'].unique():
        participant_rows = pred_data[pred_data['participant_id'] == participant_id]
        
        for _, row in participant_rows.iterrows():
            prob_est = row.get('probability_estimate')
            travel_choice = row.get('travel_choice')
            phase = row.get('phase', 'unknown')
            condition = row.get('condition_name', 'unknown')
            
            if pd.notna(prob_est) and pd.notna(travel_choice) and travel_choice in ['City A', 'City B']:
                # Check for inconsistency
                is_inconsistent = False
                inconsistency_type = None
                
                if travel_choice == 'City B' and prob_est < 50:
                    # Participant chooses City B but thinks City A has lower probability
                    is_inconsistent = True
                    inconsistency_type = 'chose_city_b_but_low_prob_cityA'
                elif travel_choice == 'City A' and prob_est > 50:
                    # Participant chooses City A but thinks City A has higher probability  
                    is_inconsistent = True
                    inconsistency_type = 'chose_city_a_but_high_prob_cityA'
                
                if is_inconsistent:
                    inconsistent_participants.append({
                        'participant_id': participant_id,
                        'phase': phase,
                        'condition': condition,
                        'travel_choice': travel_choice,
                        'probability_estimate': prob_est,
                        'inconsistency_type': inconsistency_type
                    })
    
    return pd.DataFrame(inconsistent_participants)

def filter_consistent_participants(df, verbose=True):
    """
    Filter out participants with inconsistent responses
    
    Args:
        df: Original dataframe
        verbose: Print filtering details
        
    Returns:
        Filtered dataframe with consistent participants only
    """
    # Identify inconsistent participants
    inconsistent_df = identify_inconsistent_participants(df)
    
    if len(inconsistent_df) > 0:
        inconsistent_participant_ids = inconsistent_df['participant_id'].unique()
        
        if verbose:
            print(f"Found {len(inconsistent_participant_ids)} participants with inconsistent responses:")
            for participant_id in inconsistent_participant_ids:
                participant_issues = inconsistent_df[inconsistent_df['participant_id'] == participant_id]
                print(f"  {participant_id}:")
                for _, issue in participant_issues.iterrows():
                    print(f"    Phase {issue['phase']}: chose {issue['travel_choice']} but estimated P(City A > City B) = {issue['probability_estimate']}%")
        
        # Filter out inconsistent participants
        filtered_df = df[~df['participant_id'].isin(inconsistent_participant_ids)].copy()
        
        if verbose:
            original_participants = df['participant_id'].nunique()
            filtered_participants = filtered_df['participant_id'].nunique()
            print(f"\nFiltering results:")
            print(f"  Original participants: {original_participants}")
            print(f"  Inconsistent participants removed: {len(inconsistent_participant_ids)}")
            print(f"  Remaining participants: {filtered_participants}")
        
        return filtered_df, inconsistent_df
    else:
        if verbose:
            print("No inconsistent participants found.")
        return df, pd.DataFrame()

print("Consistency checking functions defined.")

# Humidity Prediction Study Analysis

This notebook analyzes experimental data from the trust and uncertainty visualization study.
The study examines how different visualization conditions affect user trust, confidence, and decision-making across two phases.

In [ ]:
import pandas as pd
import numpy as np
import glob
import os
from pathlib import Path
import json
import warnings
import math
warnings.filterwarnings('ignore')

# Import visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Circle
import matplotlib.patches as mpatches

# Set plotting style
plt.style.use('default')
sns.set_palette("Set2")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

## 1. Data Loading and Preprocessing

In [ ]:
# Load all CSV files from the data directory
data_dir = Path('./data')
csv_files = list(data_dir.glob('user_*.csv'))
print(f"Found {len(csv_files)} participant data files:")
for file in csv_files:
    print(f"  - {file.name}")

In [ ]:
# Function to load and clean individual participant data
def load_participant_data(file_path):
    """Load a single participant CSV file and clean the data"""
    try:
        df = pd.read_csv(file_path)
        
        # Extract participant ID from filename if not in data
        if 'participant_id' not in df.columns or df['participant_id'].isna().all():
            participant_id = file_path.stem.split('_')[1]  # Extract from filename
            df['participant_id'] = participant_id
        
        # Clean condition IDs and names
        if 'condition_id' in df.columns:
            df['condition_id'] = df['condition_id'].fillna('unknown')
        
        return df
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

# Load all participant data
all_data = []
for file_path in csv_files:
    participant_data = load_participant_data(file_path)
    if participant_data is not None:
        all_data.append(participant_data)
        print(f"Loaded data for participant: {participant_data['participant_id'].iloc[0] if not participant_data['participant_id'].isna().all() else 'unknown'}")

# Combine all participant data
if all_data:
    combined_data = pd.concat(all_data, ignore_index=True)
    print(f"\nCombined dataset shape (before filtering): {combined_data.shape}")
    
    # Filter out test participants
    test_participants = ['test', 'Test', 'TEST']
    before_count = combined_data['participant_id'].nunique()
    combined_data = combined_data[~combined_data['participant_id'].isin(test_participants)]
    combined_data = combined_data[combined_data['participant_id'].notna()]  # Also remove NaN participants
    after_count = combined_data['participant_id'].nunique()
    
    print(f"Filtered out test participants and NaN entries")
    print(f"Participants: {before_count} → {after_count}")
    print(f"Final dataset shape: {combined_data.shape}")
else:
    print("No data loaded successfully")

In [ ]:
# Examine the data structure
print("Column names:")
print(combined_data.columns.tolist())
print(f"\nDataset shape: {combined_data.shape}")
# print(f"\nUnique trial types:")
# print(combined_data['trial_type'].value_counts())

In [ ]:
# Filter for relevant trial types (prediction tasks and surveys)
relevant_trials = combined_data[
    combined_data['trial_type'].isin([
        'prediction-task', 'vis-literacy', 'trust-survey', 
        'personality-survey', 'survey-text', 'survey-multi-choice'
    ])
].copy()

print(f"Filtered dataset shape: {relevant_trials.shape}")
print(f"\nTrial types in filtered data:")
print(relevant_trials['trial_type'].value_counts())

In [ ]:
# Examine condition distribution
print("Unique conditions:")
condition_counts = relevant_trials['condition_id'].value_counts(dropna=False)
print(condition_counts)

print("\nCondition names:")
condition_names = relevant_trials[['condition_id', 'condition_name']].drop_duplicates().dropna()
for _, row in condition_names.iterrows():
    print(f"  {row['condition_id']}: {row['condition_name']}")

In [ ]:
# Separate Phase 1 and Phase 2 data
prediction_data = relevant_trials[relevant_trials['trial_type'] == 'prediction-task'].copy()

# Phase separation logic
phase1_data = prediction_data[prediction_data['phase'] == 1].copy()
phase2_data = prediction_data[prediction_data['phase'] == 2].copy()

print(f"Phase 1 data: {len(phase1_data)} rows")
print(f"Phase 2 data: {len(phase2_data)} rows")

# Get visualization literacy data
vis_literacy_data = relevant_trials[relevant_trials['trial_type'] == 'vis-literacy'].copy()
print(f"Visualization literacy data: {len(vis_literacy_data)} rows")

# Separate different types of survey data

# Get trust-survey data (interface interaction and visualization trust questions)
all_trust_surveys = relevant_trials[relevant_trials['trial_type'] == 'trust-survey'].copy()
print(f"All trust survey data: {len(all_trust_surveys)} rows")

# Get personality survey data (demographics questions)
all_personality_surveys = relevant_trials[relevant_trials['trial_type'] == 'personality-survey'].copy()
print(f"All personality survey data: {len(all_personality_surveys)} rows")

# Function to check if a row represents a specific survey type based on question_order
def get_survey_type(row):
    """Determine survey type based on question_order content"""
    if 'question_order' in row and pd.notna(row['question_order']):
        try:
            question_order = eval(row['question_order'])  # Convert string to list
            if isinstance(question_order, list) and len(question_order) > 0:
                first_question = question_order[0]
                
                # Define the first question of each survey type
                if first_question == 'navigation_control':
                    return 'interaction'
                elif first_question == 'skeptical_rating':
                    return 'trust'
                elif first_question == 'respect_others':
                    return 'demographics'
        except:
            pass
    return 'unknown'

# Add survey type classification to trust surveys
all_trust_surveys['survey_type'] = all_trust_surveys.apply(get_survey_type, axis=1)

# Separate the data based on survey types
interaction_data = all_trust_surveys[all_trust_surveys['survey_type'] == 'interaction'].copy()
trust_data = all_trust_surveys[all_trust_surveys['survey_type'] == 'trust'].copy()
demographics_data = all_personality_surveys.copy()  # Demographics is in personality-survey trial type

print(f"Interaction data: {len(interaction_data)} rows")
print(f"Trust data: {len(trust_data)} rows") 
print(f"Demographics data: {len(demographics_data)} rows")

# Show survey type distribution
print(f"\nTrust survey type distribution:")
print(all_trust_surveys['survey_type'].value_counts())

## 2. Basic Statistics Tables by Condition

Each table shows conditions as rows and response variables as columns, with participant response lists in each cell.

In [ ]:
def create_condition_response_table(data, response_columns, title="Response Table"):
    """
    Create a table where each row is a condition and each column is a response variable.
    Each cell contains a list of participant responses.
    """
    # Get unique conditions
    conditions = sorted(data['condition_id'].dropna().unique())
    
    # Initialize results dictionary
    results = {}
    
    for condition in conditions:
        condition_data = data[data['condition_id'] == condition]
        condition_responses = {}
        
        for col in response_columns:
            if col in condition_data.columns:
                responses = condition_data[col].dropna().tolist()
                condition_responses[col] = responses
            else:
                condition_responses[col] = []
        
        results[condition] = condition_responses
    
    # Convert to DataFrame
    df = pd.DataFrame(results).T
    
    print(f"\n{title}")
    print("=" * len(title))
    return df

# Function to display response summary statistics
def display_response_summary(df, title="Summary"):
    """
    Display summary statistics for response lists in each cell
    """
    print(f"\n{title} - Response Counts and Basic Stats")
    print("-" * (len(title) + 30))
    
    for condition in df.index:
        print(f"\nCondition: {condition}")
        for col in df.columns:
            responses = df.loc[condition, col]
            if isinstance(responses, list) and responses:
                numeric_responses = [r for r in responses if isinstance(r, (int, float)) and not pd.isna(r)]
                if numeric_responses:
                    print(f"  {col}: n={len(numeric_responses)}, mean={np.mean(numeric_responses):.2f}, responses={numeric_responses}")
                else:
                    print(f"  {col}: n={len(responses)}, responses={responses[:5]}{'...' if len(responses) > 5 else ''}")
            else:
                print(f"  {col}: No responses")
    
    return df

### Phase 1 Responses (Baseline - No Visualization)

In [ ]:
# Phase 1 response columns
phase1_columns = ['probability_estimate', 'confidence_rating', 'travel_choice']

# Create Phase 1 table
phase1_table = create_condition_response_table(
    phase1_data, 
    phase1_columns, 
    "Phase 1 Responses (No Visualization)"
)

# Display the table
display_response_summary(phase1_table, "Phase 1 Summary")

### Phase 2 Responses (With Visualization)

In [ ]:
# Phase 2 response columns
phase2_columns = ['probability_estimate', 'confidence_rating', 'travel_choice', 'data_trust', 'skeptical_rating']

# Create Phase 2 table
phase2_table = create_condition_response_table(
    phase2_data, 
    phase2_columns, 
    "Phase 2 Responses (With Visualization)"
)

# Display the table
display_response_summary(phase2_table, "Phase 2 Summary")

In [ ]:
# Create combined dot plot visualization with probability estimates by condition
# Opacity based on confidence rating

def create_probability_dot_plot():
    """Create a dot plot showing probability estimates by condition with confidence-based opacity"""
    
    # Define condition mapping
    condition_names = {
        0: "Historical Only",
        1: "Baseline", 
        2: "PI Plot",
        3: "Ensemble Plot",
        4: "Ensemble + Hover", 
        5: "PI Plot + Hover",
        6: "PI → Ensemble",
        7: "Buggy Control",
        8: "Bad Control",
        9: "Combined PI + Ensemble"
    }
    
    # Combine Phase 1 and Phase 2 data for visualization
    viz_data = []
    
    # Add Phase 1 data
    for _, row in phase1_data.iterrows():
        if pd.notna(row['probability_estimate']) and pd.notna(row['confidence_rating']):
            viz_data.append({
                'condition_id': row['condition_id'],
                'probability_estimate': row['probability_estimate'],
                'confidence_rating': row['confidence_rating'],
                'phase': 'Phase 1'
            })
    
    # Add Phase 2 data
    for _, row in phase2_data.iterrows():
        if pd.notna(row['probability_estimate']) and pd.notna(row['confidence_rating']):
            viz_data.append({
                'condition_id': row['condition_id'],
                'probability_estimate': row['probability_estimate'],
                'confidence_rating': row['confidence_rating'],
                'phase': 'Phase 2'
            })
    
    if not viz_data:
        print("No data available for visualization")
        return
    
    viz_df = pd.DataFrame(viz_data)
    
    # Extract condition numbers for x-axis ordering
    def extract_condition_number(condition_id):
        if pd.isna(condition_id) or condition_id == 'unknown':
            return -1
        try:
            # Extract number from condition_X_name format
            parts = str(condition_id).split('_')
            if len(parts) >= 2:
                return int(parts[1])
        except:
            pass
        return -1
    
    viz_df['condition_number'] = viz_df['condition_id'].apply(extract_condition_number)
    viz_df = viz_df[viz_df['condition_number'] >= 0]  # Remove unknown conditions
    
    # Normalize confidence rating to 0-1 for opacity (1-7 scale -> 0-1 scale)
    viz_df['opacity'] = (viz_df['confidence_rating'] - 1) / 6  # Convert 1-7 to 0-1
    
    # Create the combined plot
    fig, ax = plt.subplots(1, 1, figsize=(14, 8))
    
    # Plot Phase 1 and Phase 2 data with different colors
    phase1_viz = viz_df[viz_df['phase'] == 'Phase 1']
    phase2_viz = viz_df[viz_df['phase'] == 'Phase 2']
    
    if len(phase1_viz) > 0:
        ax.scatter(phase1_viz['condition_number'], 
                  phase1_viz['probability_estimate'],
                  alpha=phase1_viz['opacity'],
                  s=120, c='blue', edgecolors='black', linewidth=1,
                  label='Phase 1 (No Forecast)')
    
    if len(phase2_viz) > 0:
        ax.scatter(phase2_viz['condition_number'], 
                  phase2_viz['probability_estimate'],
                  alpha=phase2_viz['opacity'],
                  s=120, c='red', edgecolors='black', linewidth=1,
                  label='Phase 2 (With Forecast)')
    
    # Set up the plot
    ax.set_title('Probability Estimates by Condition\n(Dot opacity represents confidence rating)', 
                fontsize=14, fontweight='bold')
    ax.set_xlabel('Experimental Condition', fontsize=12)
    ax.set_ylabel('Probability Estimate (%)', fontsize=12)
    ax.set_ylim(0, 100)
    ax.grid(True, alpha=0.3)
    
    # Force x-axis to show all conditions 0-9
    ax.set_xlim(-0.5, 9.5)
    ax.set_xticks(range(10))
    
    # Create condition labels for x-axis
    condition_labels = []
    for i in range(10):
        if i in condition_names:
            # Wrap long names for better display
            name = condition_names[i]
            if len(name) > 12:
                words = name.split()
                if len(words) > 1:
                    mid = len(words) // 2
                    name = ' '.join(words[:mid]) + '\n' + ' '.join(words[mid:])
            condition_labels.append(f"{i}\n{name}")
        else:
            condition_labels.append(f"{i}\n(No Data)")
    
    ax.set_xticklabels(condition_labels, fontsize=9, ha='center')
    
    # Add legend
    ax.legend(loc='upper right', fontsize=10)
    
    # Add opacity explanation
    fig.text(0.5, 0.02, 'Dot opacity: Low confidence (transparent) → High confidence (opaque)', 
             ha='center', fontsize=10, style='italic')
    
    plt.tight_layout()
    plt.subplots_adjust(bottom=0.15)
    plt.show()
    
    # Print summary statistics
    print(f"\nVisualization Data Summary:")
    print(f"Phase 1: {len(phase1_viz)} data points")
    print(f"Phase 2: {len(phase2_viz)} data points")
    print(f"Conditions with data: {sorted(viz_df['condition_number'].unique())}")
    
    # Show condition mapping
    print(f"\nCondition Mapping:")
    present_conditions = sorted(viz_df['condition_number'].unique())
    for i in range(10):
        status = "✓" if i in present_conditions else "✗"
        name = condition_names.get(i, "Unknown")
        print(f"  {status} Condition {i}: {name}")
    
    return viz_df

# Create the visualization
viz_summary = create_probability_dot_plot()

### Trust and Usability Measures by Condition

In [ ]:
# Define question structures for each survey type

interactionQuestions = [
    {
        "prompt": "I was in control of my navigation through this interface.",
        "labels": ["Strongly Disagree", "Disagree", "Slightly Disagree", "Neutral", 
                   "Slightly Agree", "Agree", "Strongly Agree"],
        "type": "navigation_control"
    },
    {
        "prompt": "I had some control over the content of this interface that I wanted to see.",
        "labels": ["Strongly Disagree", "Disagree", "Slightly Disagree", "Neutral", 
                   "Slightly Agree", "Agree", "Strongly Agree"],
        "type": "content_control"
    },
    {
        "prompt": "I was in control over the pace of my visit to this interface.",
        "labels": ["Strongly Disagree", "Disagree", "Slightly Disagree", "Neutral", 
                   "Slightly Agree", "Agree", "Strongly Agree"],
        "type": "pace_control"
    },
    {
        "prompt": "I could communicate with the company directly for further questions about the company or its products if I wanted to.",
        "labels": ["Strongly Disagree", "Disagree", "Slightly Disagree", "Neutral", 
                   "Slightly Agree", "Agree", "Strongly Agree"],
        "type": "company_communication"
    },
    {
        "prompt": "The interface had the ability to respond to my specific questions quickly and efficiently.",
        "labels": ["Strongly Disagree", "Disagree", "Slightly Disagree", "Neutral", 
                   "Slightly Agree", "Agree", "Strongly Agree"],
        "type": "interface_responsiveness"
    },
    {
        "prompt": "I could communicate in real time with other customers who shared my interest in this interface.",
        "labels": ["Strongly Disagree", "Disagree", "Slightly Disagree", "Neutral", 
                   "Slightly Agree", "Agree", "Strongly Agree"],
        "type": "customer_communication"
    },
    {
        "prompt": "I felt I just had a personal conversation with a sociable, knowledgeable and warm representative from the company.",
        "labels": ["Strongly Disagree", "Disagree", "Slightly Disagree", "Neutral", 
                   "Slightly Agree", "Agree", "Strongly Agree"],
        "type": "personal_conversation"
    },
    {
        "prompt": "The interface was like talking back to me while I clicked through the interface.",
        "labels": ["Strongly Disagree", "Disagree", "Slightly Disagree", "Neutral", 
                   "Slightly Agree", "Agree", "Strongly Agree"],
        "type": "interface_interaction"
    },
    {
        "prompt": "I perceived the interface to be sensitive to my needs for product information.",
        "labels": ["Strongly Disagree", "Disagree", "Slightly Disagree", "Neutral", 
                   "Slightly Agree", "Agree", "Strongly Agree"],
        "type": "interface_sensitivity"
    }
]

visualizationTrustQuestions = [
    {
        "prompt": "I was skeptical about the information presented in this visualization.",
        "labels": ["Strongly Disagree", "Disagree", "Slightly Disagree", "Neutral", 
                   "Slightly Agree", "Agree", "Strongly Agree"],
        "type": "skeptical_rating"
    },
    {
        "prompt": "I trusted this data.",
        "labels": ["Strongly Disagree", "Disagree", "Slightly Disagree", "Neutral", 
                   "Slightly Agree", "Agree", "Strongly Agree"],
        "type": "data_trust"
    },
    {
        "prompt": "I found this visualization difficult to use.",
        "labels": ["Strongly Disagree", "Disagree", "Slightly Disagree", "Neutral", 
                   "Slightly Agree", "Agree", "Strongly Agree"],
        "type": "usability_difficulty"
    },
    {
        "prompt": "I found this visualization easy to understand.",
        "labels": ["Strongly Disagree", "Disagree", "Slightly Disagree", "Neutral", 
                   "Slightly Agree", "Agree", "Strongly Agree"],
        "type": "comprehension_ease"
    }
]

personalityQuestions = [
    {
        "prompt": "I respect others.",
        "labels": ["Strongly Disagree", "Disagree", "Slightly Disagree", 
                   "Slightly Agree", "Agree", "Strongly Agree"],
        "type": "respect_others"
    },
    {
        "prompt": "I have a good word for everyone.",
        "labels": ["Strongly Disagree", "Disagree", "Slightly Disagree", 
                   "Slightly Agree", "Agree", "Strongly Agree"],
        "type": "good_word_everyone"
    },
    {
        "prompt": "I retreat from others.",
        "labels": ["Strongly Disagree", "Disagree", "Slightly Disagree", 
                   "Slightly Agree", "Agree", "Strongly Agree"],
        "type": "retreat_from_others"
    },
    {
        "prompt": "I avoid contacts with others.",
        "labels": ["Strongly Disagree", "Disagree", "Slightly Disagree", 
                   "Slightly Agree", "Agree", "Strongly Agree"],
        "type": "avoid_contacts"
    }
]

## 4. Trust and Usability Visualizations

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

def create_sectioned_trust_plots(data, condition_names, questions, section_title="Survey Results"):
    """
    Create figures with subplots for all questions in the provided question list.
    Each subplot shows ratings by condition for that question.
    
    Args:
        data: DataFrame with survey response data
        condition_names: Dict mapping condition numbers to names
        questions: List of question dictionaries with 'type' and 'prompt' keys
        section_title: Title for the overall figure
    """
    if not questions:  # Skip if no questions provided
        print(f"No questions provided for {section_title}")
        return
        
    n = len(questions)
    fig, axes = plt.subplots(nrows=n, ncols=1, figsize=(16, 4*n), sharex=True)
    if n == 1:
        axes = [axes]

    for ax, q in zip(axes, questions):
        measure_key = q["type"]
        measure_name = q["prompt"]

        # Filter data for this measure
        measure_data = data[pd.notna(data[measure_key]) & pd.notna(data["condition_id"])]

        values, cond_nums = [], []

        for _, row in measure_data.iterrows():
            try:
                cond_num = int(str(row["condition_id"]).split("_")[1])
            except:
                continue
            if cond_num == 0:
                continue
            values.append(row[measure_key])
            cond_nums.append(cond_num)

        if len(values) > 0:
            ax.scatter(cond_nums, values, alpha=0.7, s=120, edgecolors='black', color="#1f77b4")

            # Plot mean per condition
            df_plot = pd.DataFrame({"cond": cond_nums, "val": values})
            cond_means = df_plot.groupby("cond")["val"].mean()
            for c, mean_val in cond_means.items():
                ax.plot([c-0.3, c+0.3], [mean_val, mean_val], "r-", linewidth=3, alpha=0.8)

        ax.set_title(measure_name, fontsize=12, fontweight="bold", wrap=True)
        ax.set_ylim(0.5, 7.5)
        ax.set_xlim(0.5, 9.5)
        ax.set_xticks(range(1, 10))

        # X-axis labels
        labels = []
        for j in range(1, 10):
            name = condition_names.get(j, "Unknown")
            if len(name) > 10 and " " in name:
                words = name.split()
                mid = len(words) // 2
                name = " ".join(words[:mid]) + "\n" + " ".join(words[mid:])
            labels.append(f"{j}\n{name}")
        ax.set_xticklabels(labels, fontsize=9)

        ax.set_xlabel("Experimental Condition", fontsize=10)
        ax.set_ylabel("Rating (1–7)", fontsize=10)
        ax.grid(True, alpha=0.3)

    fig.suptitle(section_title, fontsize=16, fontweight="bold", y=0.95)
    plt.tight_layout()
    plt.subplots_adjust(top=0.92)
    plt.show()

In [ ]:
# Extract survey responses from JSON response column
import json

def extract_survey_responses(data):
    """Extract survey responses from the JSON response column and add as individual columns"""
    data_copy = data.copy()
    
    for idx, row in data_copy.iterrows():
        if 'response' in row and pd.notna(row['response']):
            try:
                # Parse the JSON response data
                response_data = json.loads(row['response'])
                
                # Add each response as a new column
                for key, value in response_data.items():
                    data_copy.at[idx, key] = value
                    
            except Exception as e:
                print(f"Error parsing response for row {idx}: {e}")
    
    return data_copy

print("=== EXTRACTING SURVEY RESPONSE DATA ===")

# Extract responses for each dataset
print("Extracting interaction data responses...")
interaction_data_expanded = extract_survey_responses(interaction_data)

print("Extracting trust data responses...")  
trust_data_expanded = extract_survey_responses(trust_data)

print("Extracting demographics data responses...")
demographics_data_expanded = extract_survey_responses(demographics_data)

# Check what columns are now available
print(f"\n=== AFTER EXTRACTION ===")
print(f"Interaction data shape: {interaction_data_expanded.shape}")
print("Interaction data columns:")
interaction_cols = [col for col in interaction_data_expanded.columns if col in ['navigation_control', 'content_control', 'pace_control', 'company_communication', 'interface_responsiveness', 'customer_communication', 'personal_conversation', 'interface_interaction', 'interface_sensitivity']]
print(interaction_cols)

print(f"\nTrust data shape: {trust_data_expanded.shape}")
print("Trust data columns:")
trust_cols = [col for col in trust_data_expanded.columns if col in ['skeptical_rating', 'data_trust', 'usability_difficulty', 'comprehension_ease']]
print(trust_cols)

print(f"\nDemographics data shape: {demographics_data_expanded.shape}")
print("Demographics data columns:")
demo_cols = [col for col in demographics_data_expanded.columns if col in ['respect_others', 'good_word_everyone', 'retreat_from_others', 'avoid_contacts']]
print(demo_cols)

# Update the global variables to use the expanded data
interaction_data = interaction_data_expanded
trust_data = trust_data_expanded  
demographics_data = demographics_data_expanded

In [ ]:
# Generate three separate visualizations for each data type
condition_names = {
    0: "Historical Only",
    1: "Baseline", 
    2: "PI Plot",
    3: "Ensemble Plot",
    4: "Ensemble + Hover", 
    5: "PI Plot + Hover",
    6: "PI → Ensemble",
    7: "Buggy Control",
    8: "Bad Control",
    9: "Combined PI + Ensemble"
}

# Create visualization for interaction data
print("Creating visualization for interface interaction data...")
create_sectioned_trust_plots(
    data=interaction_data,
    condition_names=condition_names,
    questions=interactionQuestions,
    section_title="Section 1: Interface Interaction & Control"
)

# Create visualization for trust data  
print("\nCreating visualization for trust/usability data...")
create_sectioned_trust_plots(
    data=trust_data,
    condition_names=condition_names,
    questions=visualizationTrustQuestions,
    section_title="Section 2: Visualization-Specific Trust"
)

In [ ]:
# ---- CONFIGURE YOUR LIKERT QUESTION COLUMNS ---- #
# interaction_cols = [
#     'navigation_control', 'content_control', 'pace_control',
#     'company_communication', 'interface_responsiveness',
#     'customer_communication', 'personal_conversation',
#     'interface_interaction', 'interface_sensitivity'
# ]

# trust_cols = [
#     'skeptical_rating', 'data_trust', 'usability_difficulty',
#     'comprehension_ease'
# ]

# demo_cols = [
#     'respect_others', 'good_word_everyone',
#     'retreat_from_others', 'avoid_contacts'
# ]

# Combine all Likert-scale columns
all_question_cols = interaction_cols + trust_cols + demo_cols

# ---- MERGE ALL DATASETS ON PARTICIPANT ID ---- #
# Replace "participant_id" with your ID field name if different
merged = (
    interaction_data.merge(trust_data, on='participant_id', how='outer')
                    .merge(demographics_data, on='participant_id', how='outer')
)

# Keep only participant + question columns
heatmap_df = merged[['participant_id'] + all_question_cols].set_index('participant_id')

# Convert all values to numeric (some may be strings)
heatmap_df = heatmap_df.apply(pd.to_numeric, errors='coerce')

# ---- CREATE HEATMAP ---- #

plt.figure(figsize=(14, 8))

plt.imshow(heatmap_df, aspect='auto', cmap='Greens', vmin=0, vmax=8)

plt.colorbar(label="Response (1 = light, 7 = dark)")
plt.title("Survey Responses Heatmap (1–7 Likert)")

# Tick labels
plt.xticks(ticks=np.arange(len(heatmap_df.columns)), labels=heatmap_df.columns, rotation=90)
plt.yticks(ticks=np.arange(len(heatmap_df.index)), labels=heatmap_df.index)

plt.tight_layout()
plt.show()

In [ ]:
# Calculate averaged interaction scores by condition
import numpy as np
from scipy import stats

# Define interaction question columns
interaction_cols = ['navigation_control', 'content_control', 'pace_control', 
                   'company_communication', 'interface_responsiveness', 
                   'customer_communication', 'personal_conversation', 
                   'interface_interaction', 'interface_sensitivity']

print("=== INTERACTION SCORES BY CONDITION ===")
print("\nCalculating averaged interaction scores (1-7 scale)...")

# Calculate average interaction scores for each participant
interaction_results = []

for _, row in interaction_data.iterrows():
    condition_id = row['condition_id']
    
    # Extract condition number
    try:
        condition_num = int(str(condition_id).split('_')[1])
    except:
        continue
        
    if condition_num == 0:  # Skip condition 0
        continue
    
    # Get all interaction scores for this participant
    scores = []
    for col in interaction_cols:
        if col in row and pd.notna(row[col]):
            scores.append(row[col])
    
    if len(scores) > 0:
        avg_score = np.mean(scores)
        interaction_results.append({
            'condition_id': condition_id,
            'condition_num': condition_num,
            'participant_id': row['participant_id'],
            'avg_interaction_score': avg_score,
            'num_questions': len(scores)
        })

# Convert to DataFrame
results_df = pd.DataFrame(interaction_results)

if len(results_df) > 0:
    # Calculate statistics by condition
    condition_stats = results_df.groupby('condition_num').agg({
        'avg_interaction_score': ['mean', 'std', 'count'],
        'condition_id': 'first'  # Get the condition ID for reference
    }).round(3)
    
    # Flatten column names
    condition_stats.columns = ['mean_score', 'std_score', 'n_participants', 'condition_id']
    
    # Calculate standard error
    condition_stats['se_score'] = (condition_stats['std_score'] / 
                                  np.sqrt(condition_stats['n_participants'])).round(3)
    
    # Add condition names
    condition_names = {
        1: "Baseline", 
        2: "PI Plot",
        3: "Ensemble Plot",
        4: "Ensemble + Hover", 
        5: "PI Plot + Hover",
        6: "PI → Ensemble",
        7: "Buggy Control",
        8: "Bad Control",
        9: "Combined PI + Ensemble"
    }
    
    condition_stats['condition_name'] = condition_stats.index.map(condition_names)
    
    # Sort by mean score (high to low)
    condition_stats_ranked = condition_stats.sort_values('mean_score', ascending=False)
    
    print("\n=== INTERACTION SCORES RANKED BY CONDITION (High to Low) ===")
    print("Format: Condition | Mean ± SE | (n participants)")
    print("-" * 65)
    
    for rank, (condition_num, row) in enumerate(condition_stats_ranked.iterrows(), 1):
        condition_name = row['condition_name']
        mean_score = row['mean_score']
        se_score = row['se_score']
        n = int(row['n_participants'])
        
        print(f"{rank:2d}. Condition {condition_num} ({condition_name:18s}) | "
              f"{mean_score:.2f} ± {se_score:.2f} | (n={n})")
    
    # Display detailed breakdown
    print(f"\n=== DETAILED STATISTICS ===")
    display_cols = ['condition_name', 'mean_score', 'std_score', 'se_score', 'n_participants']
    detailed_stats = condition_stats_ranked[display_cols].copy()
    detailed_stats.index.name = 'Condition'
    print(detailed_stats.to_string())
    
    # Show individual participant scores for verification
    print(f"\n=== INDIVIDUAL PARTICIPANT SCORES ===")
    for condition_num in sorted(condition_stats_ranked.index):
        condition_name = condition_names.get(condition_num, "Unknown")
        participant_scores = results_df[results_df['condition_num'] == condition_num]['avg_interaction_score'].tolist()
        print(f"Condition {condition_num} ({condition_name}): {participant_scores}")
    
else:
    print("No interaction data found for analysis")

In [ ]:

# --- Sort conditions by mean score (high → low) ---
condition_stats_sorted = condition_stats.sort_values("mean_score", ascending=False)

# Extract plotting data
labels = condition_stats_sorted['condition_name'].tolist()
means = condition_stats_sorted['mean_score'].tolist()
errors = condition_stats_sorted['se_score'].tolist()

plt.figure(figsize=(10, 6))

# Horizontal bar chart with error bars
plt.barh(range(len(labels)), means, xerr=errors, capsize=5)
plt.gca().invert_yaxis()  # Highest score at top

plt.yticks(range(len(labels)), labels)
plt.xlabel("Average Interaction Score")
plt.title("Interaction Scores by Condition (Mean ± SE)")

plt.tight_layout()
plt.show()

In [ ]:
# Calculate Phase 2 probability estimates and AQI estimates by condition
import numpy as np

print("=== PHASE 2 PREDICTION ESTIMATES BY CONDITION ===")
print("\nCalculating probability estimates and AQI estimates from Phase 2 data...")

# Calculate prediction estimates for each participant in Phase 2
prediction_results = []

for _, row in phase2_data.iterrows():
    condition_id = row['condition_id']
    
    # Extract condition number
    try:
        condition_num = int(str(condition_id).split('_')[1])
    except:
        continue
        
    if condition_num == 0:  # Skip condition 0 (historical only)
        continue
    
    # Get prediction estimates
    probability_est = row.get('probability_estimate')
    city_a_est = row.get('city_a_estimate') 
    city_b_est = row.get('city_b_estimate')
    confidence = row.get('confidence_rating')
    
    if pd.notna(probability_est):  # At minimum need probability estimate
        prediction_results.append({
            'condition_id': condition_id,
            'condition_num': condition_num,
            'participant_id': row['participant_id'],
            'probability_estimate': probability_est,
            'city_a_estimate': city_a_est if pd.notna(city_a_est) else None,
            'city_b_estimate': city_b_est if pd.notna(city_b_est) else None,
            'confidence_rating': confidence if pd.notna(confidence) else None
        })

# Convert to DataFrame
pred_results_df = pd.DataFrame(prediction_results)

if len(pred_results_df) > 0:
    # Calculate statistics by condition for each measure
    print("\n=== PROBABILITY ESTIMATES (City A > City B) ===")
    prob_stats = pred_results_df.groupby('condition_num').agg({
        'probability_estimate': ['mean', 'std', 'count'],
        'condition_id': 'first'
    }).round(2)
    prob_stats.columns = ['mean_prob', 'std_prob', 'n_prob', 'condition_id']
    prob_stats['se_prob'] = (prob_stats['std_prob'] / np.sqrt(prob_stats['n_prob'])).round(2)
    
    # City A estimates
    print("\n=== CITY A AQI ESTIMATES ===") 
    city_a_data = pred_results_df[pred_results_df['city_a_estimate'].notna()]
    if len(city_a_data) > 0:
        city_a_stats = city_a_data.groupby('condition_num').agg({
            'city_a_estimate': ['mean', 'std', 'count'],
            'condition_id': 'first'
        }).round(2)
        city_a_stats.columns = ['mean_city_a', 'std_city_a', 'n_city_a', 'condition_id']
        city_a_stats['se_city_a'] = (city_a_stats['std_city_a'] / np.sqrt(city_a_stats['n_city_a'])).round(2)
    else:
        city_a_stats = pd.DataFrame()
    
    # City B estimates  
    print("\n=== CITY B AQI ESTIMATES ===")
    city_b_data = pred_results_df[pred_results_df['city_b_estimate'].notna()]
    if len(city_b_data) > 0:
        city_b_stats = city_b_data.groupby('condition_num').agg({
            'city_b_estimate': ['mean', 'std', 'count'], 
            'condition_id': 'first'
        }).round(2)
        city_b_stats.columns = ['mean_city_b', 'std_city_b', 'n_city_b', 'condition_id']
        city_b_stats['se_city_b'] = (city_b_stats['std_city_b'] / np.sqrt(city_b_stats['n_city_b'])).round(2)
    else:
        city_b_stats = pd.DataFrame()
    
    # Combine all statistics
    combined_stats = prob_stats.copy()
    if not city_a_stats.empty:
        combined_stats = combined_stats.join(city_a_stats[['mean_city_a', 'se_city_a', 'n_city_a']], how='left')
    if not city_b_stats.empty:
        combined_stats = combined_stats.join(city_b_stats[['mean_city_b', 'se_city_b', 'n_city_b']], how='left')
    
    # Add condition names
    condition_names = {
        1: "Baseline", 
        2: "PI Plot",
        3: "Ensemble Plot",
        4: "Ensemble + Hover", 
        5: "PI Plot + Hover",
        6: "PI → Ensemble",
        7: "Buggy Control",
        8: "Bad Control",
        9: "Combined PI + Ensemble"
    }
    combined_stats['condition_name'] = combined_stats.index.map(condition_names)
    
    # Sort by probability estimate (LOW to HIGH - lower probability ranked higher)
    combined_stats_ranked = combined_stats.sort_values('mean_prob', ascending=True)
    
    print("\n" + "="*80)
    print("PHASE 2 PREDICTION ESTIMATES RANKED BY PROBABILITY (Low to High)")
    print("="*80)
    print("Format: Condition | P(City A > City B) | City A AQI | City B AQI | (n)")
    print("-" * 80)
    
    for rank, (condition_num, row) in enumerate(combined_stats_ranked.iterrows(), 1):
        condition_name = row['condition_name']
        
        # Probability estimate
        prob_mean = row['mean_prob']
        prob_se = row['se_prob']
        n_prob = int(row['n_prob'])
        
        # City A estimate
        if pd.notna(row.get('mean_city_a')):
            city_a_mean = row['mean_city_a']
            city_a_se = row['se_city_a']
            city_a_str = f"{city_a_mean:.1f} ± {city_a_se:.1f}"
        else:
            city_a_str = "N/A"
        
        # City B estimate  
        if pd.notna(row.get('mean_city_b')):
            city_b_mean = row['mean_city_b']
            city_b_se = row['se_city_b']
            city_b_str = f"{city_b_mean:.1f} ± {city_b_se:.1f}"
        else:
            city_b_str = "N/A"
        
        print(f"{rank:2d}. Condition {condition_num} ({condition_name:18s}) | "
              f"{prob_mean:5.1f}% ± {prob_se:4.1f} | {city_a_str:>12s} | {city_b_str:>12s} | (n={n_prob})")
    
    # Detailed breakdown
    print(f"\n" + "="*60)
    print("DETAILED STATISTICS TABLE")
    print("="*60)
    display_cols = ['condition_name', 'mean_prob', 'se_prob', 'n_prob']
    if not city_a_stats.empty:
        display_cols.extend(['mean_city_a', 'se_city_a'])
    if not city_b_stats.empty:
        display_cols.extend(['mean_city_b', 'se_city_b'])
    
    detailed_pred_stats = combined_stats_ranked[display_cols].copy()
    detailed_pred_stats.index.name = 'Condition'
    print(detailed_pred_stats.to_string())
    
    # Individual participant data for verification
    print(f"\n" + "="*60)
    print("INDIVIDUAL PARTICIPANT ESTIMATES")
    print("="*60)
    for condition_num in sorted(combined_stats_ranked.index):
        condition_name = condition_names.get(condition_num, "Unknown")
        condition_data = pred_results_df[pred_results_df['condition_num'] == condition_num]
        
        prob_values = condition_data['probability_estimate'].tolist()
        city_a_values = condition_data['city_a_estimate'].dropna().tolist()
        city_b_values = condition_data['city_b_estimate'].dropna().tolist()
        
        print(f"\nCondition {condition_num} ({condition_name}):")
        print(f"  Probability estimates: {prob_values}")
        if city_a_values:
            print(f"  City A AQI estimates: {city_a_values}")
        if city_b_values:
            print(f"  City B AQI estimates: {city_b_values}")
    
else:
    print("No Phase 2 prediction data found for analysis")

![alt text](SCR-20251208-ovoi-1.png)